In [5]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import torchvision.transforms as transform
from torchvision.datasets import MNIST
from tqdm import tqdm

In [12]:
train_set = MNIST('', train=True, download=True, 
                  transform=transform.Compose([transform.ToTensor(), transform.Normalize((0.1307,), (0.3081,))]))

test_set = MNIST('', train=False, download=True, 
                 transform=transform.Compose([transform.ToTensor(), transform.Normalize((0.1307,), (0.3081,))]))

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|███████████████████████████████████████████████████████████████████| 9912422/9912422 [00:09<00:00, 1075969.09it/s]


Extracting MNIST\raw\train-images-idx3-ubyte.gz to MNIST\raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|████████████████████████████████████████████████████████████████████████| 28881/28881 [00:00<00:00, 149062.75it/s]


Extracting MNIST\raw\train-labels-idx1-ubyte.gz to MNIST\raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|███████████████████████████████████████████████████████████████████| 1648877/1648877 [00:01<00:00, 1632278.31it/s]


Extracting MNIST\raw\t10k-images-idx3-ubyte.gz to MNIST\raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████████████████████████████████████████████████████████████████████████████████| 4542/4542 [00:00<?, ?it/s]

Extracting MNIST\raw\t10k-labels-idx1-ubyte.gz to MNIST\raw



In [15]:
batch_size = 32

loader = {
    "train" : DataLoader(train_set, batch_size = batch_size, shuffle = True),
    "test" : DataLoader(test_set, batch_size = batch_size, shuffle = True)
}

### FNN

In [4]:
class NN(nn.Module):
    def __init__(self):
        super().__init__()
        # Definiramo objekt, ki bo sliko spremenil v vektor
        self.flatten = nn.Flatten()
        # Definiramo sloje nevronske mreže
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 64),
            nn.ReLU(),
            nn.Linear(64, 20),
            nn.ReLU(),
            nn.Linear(20, 10)
        )
        self.softmax = nn.Softmax(dim=1)
        

    def forward(self, x):
        # Vhodno sliko pretvorimo v vektor
        x = self.flatten(x)
        # Vektor pošljemo čez vse sloje in aktivacijske vrednosti
        logits = self.linear_relu_stack(x)
        # Izhod iz nevronske mreže pretvorimo v "verjetnosti" za vsako ciljno vrednost
        return self.softmax(logits)

In [16]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = NN().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)
criterion = nn.CrossEntropyLoss()

In [18]:
def train(epochs):  
    # Zagotovimo, da bo model v načinu treniranja, kjer se računajo gradienti in so aktivni vsi sloji
    model.train()
    for epoch in range(epochs):
        running_loss = 0
        with tqdm(total = len(loader["train"]) * batch_size, desc = f'Training - Epoch: {epoch + 1}/{epochs}', unit = 'chunks') as prog_bar:
            # Gremo čez vse podatke v skupinah po batch_size z trainloaderjem, v našem primeru je to 32
            for i, data in enumerate(loader["train"], 0):
                # Podatke razpakiramo v vhode in izhode 
                inputs, labels = data
                # Resetiramo gradiente v podatkih
                optimizer.zero_grad()
                # Vhodne podatke spustimo čez model, ta nam vrne matriko, v kateri se vsaka vrstica sešteje v 1 (zaradi Softmax sloja)
                outputs = model(inputs)
                # Izračunamo izgubo
                loss = criterion(outputs, labels)
                # Naredimo vzvratno razširanje napake (backpropagation)
                loss.backward()
                # Naredimo en korak optimizacije
                optimizer.step()
                # Dodamo izgubo k naši vsoti izgube.
                running_loss += loss.detach().item()
                prog_bar.set_postfix(**{'loss': (running_loss) / (i+1)})
                prog_bar.update(batch_size)
    print('Finished Training')

In [26]:
def test():
    model.eval()
    test_loss, correct, counter = 0, 0, 0
   
    with torch.no_grad():
        with tqdm(total = len(loader["test"]) * batch_size, desc = f'Testing', unit = 'chunks') as prog_bar:
            for i, data in enumerate(loader["test"], 0):
                inputs, labels = data
                output = model(inputs)
                test_loss += criterion(output, labels).detach().item()
                # Izberemo indekse mesta z najvišjo vrednostjo ("verjetnostjo")
                pred = output.data.max(1, keepdim = True)[1]
                # Prištejemo število primerov, kjer smo zadeli pravilen rezultat
                correct += pred.eq(labels.data.view_as(pred)).sum()
                prog_bar.update(batch_size)
                counter += 1
    print(f'Test set: Avg. loss: {test_loss/counter}, Correct predictions: {correct}/{len(loader["test"].dataset)}')

In [27]:
train(5)
print("")
test()

Training - Epoch: 5/5: 100%|████████████████████████████████████| 60000/60000 [00:40<00:00, 1475.20chunks/s, loss=1.48]


Finished Training



Testing: 100%|█████████████████████████████████████████████████████████████| 10016/10016 [00:04<00:00, 2019.61chunks/s]

Test set: Avg. loss: 1.4944515479639315, Correct predictions: 9669/10000


### CNN

In [28]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        # Ker bomo delali na slikah (2d matrikah) slike ne rabimo pretvoriti v vektor
        self.convolution = nn.Sequential(
            # Začnemo s konvolucijskim slojem. Ta bo na vhod dobil 1 kanal (ker je slika črno bela)
            # na izhod pa vrnila 16 kanalov (sami definiramo koliko). Velikost konvolucijskega filtra
            # nadzorujemo z parametrom kernel_size, v našem primeru bodo filtri velikosti 3x3
            nn.Conv2d(1, 16, kernel_size=3),
            # Dobljene kanale pošljemo čez relu funkcijo
            nn.ReLU(),
            # Sliko pomanjšamo z maxpool slojem. Jedro tega bo velikosti 2 v našem primeru
            nn.MaxPool2d(kernel_size=2))

        # Na koncu bomo ciljno vrednost napovedali s polno povezanim slojem. Ta na vhod prejme kanale
        # iz prejšnjega sloja pretvorjene v vektor. Teh je 16, vsak velikosti 13x13. 
        self.fc = nn.Linear(16*13*13, 10)
    
    def forward(self, x):
        # Podatke pošljemo čez konvolucijo
        x = self.convolution(x)
        # Kanale pretvorimo v vektorje z ukazom reshape
        x = x.reshape(x.size(0), -1)
        # Napovemo ciljno vrednost s polno povezanim slojem
        x = self.fc(x)
        return x

In [29]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = CNN().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)
criterion = nn.CrossEntropyLoss()

In [30]:
train(5)
print("")
test()

Training - Epoch: 5/5: 100%|██████████████████████████████████| 60000/60000 [00:52<00:00, 1143.48chunks/s, loss=0.0419]


Finished Training



Testing: 100%|█████████████████████████████████████████████████████████████| 10016/10016 [00:06<00:00, 1605.39chunks/s]

Test set: Avg. loss: 0.060103129838948874, Correct predictions: 9808/10000
